In [1]:
import time
from os import environ

from spatialyze.database import database
from spatialyze.predicate import camera, objects, lit
from spatialyze.utils import F
from spatialyze.utils.process_raw_nuscenes import process_raw_nuscenes

In [2]:
database.load_roadnetworks('../../data/scenic/road-network/boston-seaport', 'boston-seaport')

Location: boston-seaport
Ingesting polygon
Ingesting segment
Ingesting lane
Ingesting road
Ingesting laneGroup
Ingesting laneSection
Ingesting roadSection
Ingesting intersection
Ingesting lane_LaneSec
Ingesting laneGroup_Lane
Ingesting laneGroup_opposite
Ingesting road_laneGroup
Ingesting road_roadSec
Ingesting roadSec_laneSec
altered table
added type: lanesection
added type: road
added type: lane
added type: lanegroup
added type: intersection
added type: roadsection
index created: lanesection
index created: road
index created: lane
index created: lanegroup
index created: intersection
index created: roadsection


In [3]:
database.reset()

In [4]:
cameras, annotations = process_raw_nuscenes(environ['NUSCENES_MINI'])

KeyError: 'NUSCENES_MINI'

In [ ]:
database.load_nuscenes(cameras, annotations)

## Simple Query

In [ ]:
c = camera
o = objects[0]
output = database.predicate(o.type == 'vehicle.car')

In [ ]:
len(output)

## Figure 12

In [ ]:
t0 = time.time()
o = objects[0]
c = camera
pred1 = (
    # (F.like(c.filename, 'samples/CAM_FRONT%')) & 
    (o.type == 'human.pedestrian.adult') &
    # F.contained(c.ego, 'intersection') &
    F.contained(o.trans@c.time, 'intersection') &
    F.angle_excluding(F.facing_relative(o.traj@c.time, c.cam), lit(-70), lit(70)) &
    # F.angle_between(F.facing_relative(c.cam, F.road_direction(c.ego)), lit(-15), lit(15)) &
    (F.distance(c.ego, o.traj@c.time) < lit(50)) & # &
    (F.view_angle(o.trans@c.time, c.cam) < lit(35))
)
output = database.predicate(pred1)
t1 = time.time()

In [ ]:
print(len(output), t1 - t0)

## Figure 13

In [ ]:
t0 = time.time()
obj1 = objects[0]
obj2 = objects[1]
cam = camera
pred2 =(
    # (F.like(cam.filename, 'samples/CAM_FRONT%')) & 
    (obj1.id > obj2.id) &
    ((obj1.type == 'vehicle.car') | (obj1.type == 'vehicle.truck')) &
    ((obj2.type == 'vehicle.car') | (obj2.type == 'vehicle.truck')) &
    # F.angle_between(F.facing_relative(cam.cam, F.road_direction(cam.cam)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.cam) < lit(35)) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.cam) < lit(35)) &
    F.contained(obj1.trans@cam.time, 'intersection') &
    F.contained(obj2.trans@cam.time, 'intersection') &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.cam), 40, 135) &
    # F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.cam), -135, -50) &
    F.angle_between(F.facing_relative(obj1.traj@cam.time, obj2.traj@cam.time), -180, -90)
    # (F.min_distance(cam.ego, 'intersection') < 10) &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)
output = database.predicate(pred2)
t1 = time.time()

In [ ]:
len(output)

In [ ]:
t1 - t0

## Figure 14

In [ ]:
t0 = time.time()
obj1 = objects[0]
cam = camera
pred3 = (
    # (F.like(cam.filename, 'samples/CAM_FRONT%')) & 
    ((obj1.type == 'vehicle.car') | (obj1.type == 'vehicle.truck')) &
    (F.view_angle(obj1.trans@cam.time, cam.cam) < lit(35)) &
    F.angle_between(F.facing_relative(cam.cam, F.road_direction(cam.ego, cam.ego)), 135, 225) &
    F.contained(cam.cam, F.road_segment('lane')) &
    F.contained(obj1.trans@cam.time, F.road_segment('lane')) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, F.road_direction(obj1.traj@cam.time, cam.ego)), -15, 15) &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 135, 225) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 10)
)
output = database.predicate(pred3)
t1 = time.time()

In [ ]:
print(len(output), t1 - t0)

## Figure 15

In [ ]:
t0 = time.time()
cam = camera
car1 = objects[0]
opposite_car_1 = objects[1]
opposite_car_2 = objects[2]

pred4 = (
    # (F.like(cam.filename, 'samples/CAM_FRONT%')) & 
    ((car1.type == 'vehicle.car') | (car1.type == 'vehicle.truck')) &
    ((opposite_car_2.type == 'vehicle.car') | (opposite_car_2.type == 'vehicle.truck')) &
    ((opposite_car_1.type == 'vehicle.car') | (opposite_car_1.type == 'vehicle.truck')) &
    (opposite_car_1.id > opposite_car_2.id) &
    (car1.id < opposite_car_2.id) &

    F.contained(cam.cam, F.road_segment('lane')) &
    F.contained(car1.trans@cam.time, F.road_segment('lane')) &
    F.contained(opposite_car_1.trans@cam.time, F.road_segment('lane')) &
    F.contained(opposite_car_2.trans@cam.time, F.road_segment('lane')) &
    F.angle_between(F.facing_relative(cam.cam, F.road_direction(cam.cam, cam.cam)), -15, 15) &
    (F.view_angle(car1.trans@cam.time, cam.cam) < lit(35)) &
    (F.distance(cam.cam, car1.traj@cam.time) < 40) &
    F.angle_between(F.facing_relative(car1.traj@cam.time, cam.ego), -15, 15) &
    # F.angle_between(F.facing_relative(car1.traj@cam.time, F.road_direction(car1.traj@cam.time, cam.ego)), -15, 15) &
    F.ahead(car1.traj@cam.time, cam.cam) &
    # (F.convert_camera(opposite_car.traj@cam.time, cam.ego) > [-10, 0]) &
    # (F.convert_camera(opposite_car.traj@cam.time, cam.ego) < [-1, 50]) &
    F.angle_between(F.facing_relative(opposite_car_1.traj@cam.time, cam.ego), 135, 225) &
    # (F.distance(opposite_car@cam.time, car2@cam.time) < 40)# &
    F.angle_between(F.facing_relative(opposite_car_2.traj@cam.time, opposite_car_1.traj@cam.time), -15, 15) &
    F.angle_between(F.facing_relative(opposite_car_2.traj@cam.time, F.road_direction(opposite_car_2.traj@cam.time, cam.ego)), -15, 15) &
    F.ahead(opposite_car_2.traj@cam.time, opposite_car_1.traj@cam.time)
    )
output = database.predicate(pred4)
t1 = time.time()

In [ ]:
print(len(output), t1 - t0)